In [11]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets, Sequential

x = tf.random.normal([2,5,5,3])
w = tf.random.normal([3,3,3,4])
out1 = tf.nn.conv2d(x,w,strides=1,padding=[[0,0],[0,0],[0,0],[0,0]])
out2 = tf.nn.conv2d(x,w,strides=1,padding=[[0,0],[1,1],[1,1],[0,0]])

# out size: h' = (h+ 2 * ph - k)/s + 1
# out size: w' = (w+ 2 * pw - k)/s + 1
#print(out1, out2)

out3 = tf.nn.conv2d(x,w,strides=3,padding='SAME')
#print(out3)

b = tf.zeros([4])
out1 = out1 + b
out2 = out2 + b
out3 = out3 + b

# Create Convolutional Class
layer = layers.Conv2D(4, kernel_size=(3,4), strides=(2,1),padding='SAME')
out4 = layer(x)
out4.shape

layer.kernel, layer.bias, layer.trainable_variables

(<tf.Variable 'conv2d_7/kernel:0' shape=(3, 4, 3, 4) dtype=float32, numpy=
 array([[[[ 0.20537695,  0.07819858, -0.10377975, -0.07577212],
          [ 0.25597224,  0.11768016,  0.01110002, -0.13808885],
          [ 0.15076321, -0.05737492,  0.22568199,  0.22383493]],
 
         [[-0.1695939 ,  0.14603978, -0.22955827,  0.25673482],
          [-0.21201652,  0.03745955,  0.1090647 ,  0.15632024],
          [ 0.1363835 , -0.20359084,  0.16536582,  0.08823526]],
 
         [[-0.06418075, -0.24729086,  0.12645134,  0.2506878 ],
          [ 0.2401655 , -0.14394665,  0.22118169,  0.1007911 ],
          [ 0.24422929, -0.2031453 , -0.15395705, -0.10557799]],
 
         [[ 0.08532217,  0.13762656,  0.26061866,  0.09217614],
          [ 0.2467325 ,  0.09048182, -0.09799895,  0.18177769],
          [ 0.16489175, -0.05202773,  0.10804141,  0.26527706]]],
 
 
        [[[ 0.05969039,  0.07262486, -0.1597141 ,  0.02852243],
          [ 0.06405234,  0.25725827,  0.04834339,  0.2179606 ],
          [ 0.

In [37]:
from tensorflow.keras import Sequential

# Use Sequential to create LeNet structure

network = Sequential(
    [
    layers.Conv2D(6,kernel_size=3,strides=1), #Param No: (3*3 + 1)*6 = 60
    layers.MaxPooling2D(pool_size=2, strides=2),
    layers.ReLU(),
    layers.Conv2D(16, kernel_size=3, strides = 1), #Param No: (3*3*6 + 1) *16 = 880 
    layers.MaxPooling2D(pool_size=2, strides=2),
    layers.ReLU(),
    layers.Flatten(),
    
    layers.Dense(120, activation = 'relu'), #Param No: (400+1) * 120
    layers.Dense(84, activation = 'relu'), #Param No: (120+1) * 84
    layers.Dense(10) #Param No: (84+1) * 10
    ]
)
network.build(input_shape=([4,28,28,1]))
network.summary()

from tensorflow.keras import losses, optimizers

criteon = losses.CategoricalCrossentropy(from_logits=True)
x = tf.random.normal([100,28,28])
y = tf.random.uniform([100],0,10, dtype=tf.int32)

with tf.GradientTape() as tape:
    x = tf.expand_dims(x, axis=-1)
    out = network(x)
    y_onehot = tf.one_hot(y, depth=10)
    loss = criteon(y_onehot, out)
grads = tape.gradient(loss, network.trainable_variables)
optimizer.apply_gradients(zip(grads, network.trainable_variables))

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           multiple                  60        
_________________________________________________________________
max_pooling2d_34 (MaxPooling multiple                  0         
_________________________________________________________________
re_lu_34 (ReLU)              multiple                  0         
_________________________________________________________________
conv2d_43 (Conv2D)           multiple                  880       
_________________________________________________________________
max_pooling2d_35 (MaxPooling multiple                  0         
_________________________________________________________________
re_lu_35 (ReLU)              multiple                  0         
_________________________________________________________________
flatten_17 (Flatten)         multiple                

NameError: name 'optimizer' is not defined

In [46]:
def preprocess(x, y):
    x = 2*tf.cast(x, dtype=tf.float32) / 255. - 1
    y = tf.cast(y,dtype=tf.int32)
    return x,y

(x,y),(x_test, y_test) = datasets.cifar100.load_data()
y = tf.squeeze(y, axis= -1)
y_test = tf.squeeze(y_test, axis = -1)

train_db = tf.data.Dataset.from_tensor_slices((x,y))
train_db = train_db.shuffle(1000).map(preprocess).batch(128)

test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_db = test_db.map(preprocess).batch(128)

sample = next(iter(train_db))
print('sample:', sample[0].shape, sample[1].shape, tf.reduce_min(sample[0]), tf.reduce_max(sample[0]))

conv_layers = [
    layers.Conv2D(64, kernel_size = [3,3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(64, kernel_size = [3,3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'),
    
    layers.Conv2D(128, kernel_size = [3,3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(128, kernel_size = [3,3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'),
    
    layers.Conv2D(256, kernel_size = [3,3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(256, kernel_size = [3,3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'),
    
    layers.Conv2D(512, kernel_size = [3,3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size = [3,3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same'),
    
    layers.Conv2D(512, kernel_size = [3,3], padding='same', activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size = [3,3], padding='same', activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2,2], strides=2, padding='same')
]

conv_net = Sequential(conv_layers)

fc_layers = [
    layers.Dense(256, activation=tf.nn.relu),
    layers.Dense(128, activation=tf.nn.relu),
    layers.Dense(100, activation=None)
]

fc_net = Sequential(fc_layers)

conv_net.build(input_shape=[4,32,32,3])
fc_net.build(input_shape=[4,512])
conv_net.summary()
fc_net.summary()

sample: (128, 32, 32, 3) (128,) tf.Tensor(-1.0, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32)
Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_44 (Conv2D)           multiple                  1792      
_________________________________________________________________
conv2d_45 (Conv2D)           multiple                  36928     
_________________________________________________________________
max_pooling2d_36 (MaxPooling multiple                  0         
_________________________________________________________________
conv2d_46 (Conv2D)           multiple                  73856     
_________________________________________________________________
conv2d_47 (Conv2D)           multiple                  147584    
_________________________________________________________________
max_pooling2d_37 (MaxPooling multiple                  0         
______